# Word2Vec Tutorial

In case you missed the buzz, word2vec is a widely featured as a member of the “new wave” of machine learning algorithms based on neural networks, commonly referred to as "deep learning" (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

Word2vec is very useful in [automatic text tagging](https://github.com/RaRe-Technologies/movie-plots-by-genre), recommender systems and machine translation.

Check out an [online word2vec demo](http://radimrehurek.com/2014/02/word2vec-tutorial/#app) where you can try this vector algebra for yourself. That demo runs `word2vec` on the Google News dataset, of **about 100 billion words**.

## This tutorial

In this tutorial you will learn how to train and evaluate word2vec models on your business data.


## Preparing the Input
Starting from the beginning, gensim’s `word2vec` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [1]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-25 14:55:29,166 : INFO : collecting all words and their counts
2017-06-25 14:55:29,169 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:29,172 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2017-06-25 14:55:29,173 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:29,175 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-25 14:55:29,177 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-25 14:55:29,178 : INFO : deleting the raw counts dictionary of 3 items
2017-06-25 14:55:29,180 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-25 14:55:29,181 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-25 14:55:29,183 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-25 14:55:29,184 : INFO : resetting layer weights
2017-06-25 14:55:29,253 : INFO : training model with 3 workers o

Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [3]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

In [5]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[['first'], ['sentence'], ['second'], ['sentence']]


In [6]:
# generate the Word2Vec model
model = gensim.models.Word2Vec(sentences, min_count=1)

2017-06-25 14:55:37,493 : INFO : collecting all words and their counts
2017-06-25 14:55:37,496 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:37,500 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-25 14:55:37,502 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:37,505 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-25 14:55:37,508 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-25 14:55:37,511 : INFO : deleting the raw counts dictionary of 3 items
2017-06-25 14:55:37,513 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-25 14:55:37,514 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-25 14:55:37,516 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-25 14:55:37,517 : INFO : resetting layer weights
2017-06-25 14:55:37,519 : INFO : training model with 3 workers o

In [7]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7f650a0a9e80>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7f650a0ba390>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7f650a0ba5f8>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `word2vec` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `Word2Vec(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Google's word2vec in C language. 
  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [8]:
# build the same model, making the 2 steps explicit
new_model = gensim.models.Word2Vec(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2017-06-25 14:55:40,766 : INFO : collecting all words and their counts
2017-06-25 14:55:40,769 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:40,772 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2017-06-25 14:55:40,774 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:40,776 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2017-06-25 14:55:40,778 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2017-06-25 14:55:40,780 : INFO : deleting the raw counts dictionary of 3 items
2017-06-25 14:55:40,781 : INFO : sample=0.001 downsamples 3 most-common words
2017-06-25 14:55:40,783 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2017-06-25 14:55:40,785 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2017-06-25 14:55:40,786 : INFO : resetting layer weights
2017-06-25 14:55:40,788 : INFO : training model with 3 workers o

0

In [9]:
print(new_model)
print(model.wv.vocab)

Word2Vec(vocab=3, size=100, alpha=0.025)
{'first': <gensim.models.keyedvectors.Vocab object at 0x7f650a0a9e80>, 'sentence': <gensim.models.keyedvectors.Vocab object at 0x7f650a0ba390>, 'second': <gensim.models.keyedvectors.Vocab object at 0x7f650a0ba5f8>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [10]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [11]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`Word2Vec` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [12]:
# default value of min_count=5
model = gensim.models.Word2Vec(sentences, min_count=10)

2017-06-25 14:55:46,535 : INFO : collecting all words and their counts
2017-06-25 14:55:46,659 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:46,713 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-25 14:55:46,714 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:46,729 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2017-06-25 14:55:46,731 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2017-06-25 14:55:46,738 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-25 14:55:46,740 : INFO : sample=0.001 downsamples 54 most-common words
2017-06-25 14:55:46,741 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2017-06-25 14:55:46,742 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2017-06-25 14:55:46,745 : INFO : resetting layer weights
2017-06-25 14:5

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [13]:
# default value of size=100
model = gensim.models.Word2Vec(sentences, size=200)

2017-06-25 14:55:48,341 : INFO : collecting all words and their counts
2017-06-25 14:55:48,346 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:48,409 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-25 14:55:48,411 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:48,421 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-25 14:55:48,422 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-25 14:55:48,430 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-25 14:55:48,432 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-25 14:55:48,433 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-25 14:55:48,434 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2017-06-25 14:55:48,441 : INFO : resetting layer weights
2017-06-25 14:

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [14]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(sentences, workers=4)

2017-06-25 14:55:50,815 : INFO : collecting all words and their counts
2017-06-25 14:55:50,819 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:55:50,866 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2017-06-25 14:55:50,868 : INFO : Loading a fresh vocabulary
2017-06-25 14:55:50,885 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2017-06-25 14:55:50,886 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2017-06-25 14:55:50,900 : INFO : deleting the raw counts dictionary of 10186 items
2017-06-25 14:55:50,901 : INFO : sample=0.001 downsamples 49 most-common words
2017-06-25 14:55:50,902 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2017-06-25 14:55:50,905 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2017-06-25 14:55:50,923 : INFO : resetting layer weights
2017-06-25 14:

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `word2vec` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `word2vec` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`Word2Vec` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [15]:
model.accuracy('./datasets/questions-words.txt')

2017-06-25 14:55:55,589 : INFO : precomputing L2-norms of word weight vectors
2017-06-25 14:55:55,835 : INFO : family: 0.0% (0/2)
2017-06-25 14:55:55,874 : INFO : gram3-comparative: 0.0% (0/12)
2017-06-25 14:55:55,893 : INFO : gram4-superlative: 0.0% (0/12)
2017-06-25 14:55:55,922 : INFO : gram5-present-participle: 0.0% (0/20)
2017-06-25 14:55:55,942 : INFO : gram6-nationality-adjective: 0.0% (0/20)
2017-06-25 14:55:55,972 : INFO : gram7-past-tense: 0.0% (0/20)
2017-06-25 14:55:55,986 : INFO : gram8-plural: 0.0% (0/12)
2017-06-25 14:55:55,994 : INFO : total: 0.0% (0/98)


[{'correct': [], 'incorrect': [], 'section': 'capital-common-countries'},
 {'correct': [], 'incorrect': [], 'section': 'capital-world'},
 {'correct': [], 'incorrect': [], 'section': 'currency'},
 {'correct': [], 'incorrect': [], 'section': 'city-in-state'},
 {'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')],
  'section': 'family'},
 {'correct': [], 'incorrect': [], 'section': 'gram1-adjective-to-adverb'},
 {'correct': [], 'incorrect': [], 'section': 'gram2-opposite'},
 {'correct': [],
  'incorrect': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
   ('GOOD', 'BETTER', 'LONG', 'LONGER'),
   ('GOOD', 'BETTER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'LONG', 'LONGER'),
   ('GREAT', 'GREATER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'LOW', 'LOWER'),
   ('LONG', 'LONGER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'GREAT', 'GREATER'),
   ('LOW', 'LOWER', 'GOOD', 'BETTER'),
   ('LOW', 'LOWER', 'GREAT', 'GREATER'),
   (

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



In the December 2016 release of Gensim we added a better way to evaluate semantic similarity.

By default it uses an academic dataset WS-353 but one can create a dataset specific to your business based on it. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [16]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

2017-06-25 14:55:58,340 : INFO : Pearson correlation coefficient against /home/aneeshjoshi/Anaconda3.5install/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1494
2017-06-25 14:55:58,341 : INFO : Spearman rank-order correlation coefficient against /home/aneeshjoshi/Anaconda3.5install/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: 0.1759
2017-06-25 14:55:58,342 : INFO : Pairs with unknown words ratio: 85.6%


((0.14935402024332387, 0.295542603592447),
 SpearmanrResult(correlation=0.17594750577927895, pvalue=0.21682015715863545),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [17]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2017-06-25 14:55:59,989 : INFO : saving Word2Vec object under /tmp/tmp6jh4qoa_gensim_temp, separately None
2017-06-25 14:55:59,991 : INFO : not storing attribute syn0norm
2017-06-25 14:55:59,993 : INFO : not storing attribute cum_table
2017-06-25 14:56:00,014 : INFO : saved /tmp/tmp6jh4qoa_gensim_temp


In [18]:
new_model = gensim.models.Word2Vec.load(temp_path)  # open the model

2017-06-25 14:56:00,537 : INFO : loading Word2Vec object from /tmp/tmp6jh4qoa_gensim_temp
2017-06-25 14:56:00,587 : INFO : loading wv recursively from /tmp/tmp6jh4qoa_gensim_temp.wv.* with mmap=None
2017-06-25 14:56:00,589 : INFO : setting ignored attribute syn0norm to None
2017-06-25 14:56:00,592 : INFO : setting ignored attribute cum_table to None
2017-06-25 14:56:00,597 : INFO : loaded /tmp/tmp6jh4qoa_gensim_temp


which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [19]:
model = gensim.models.Word2Vec.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2017-06-25 14:56:02,595 : INFO : loading Word2Vec object from /tmp/tmp6jh4qoa_gensim_temp
2017-06-25 14:56:02,624 : INFO : loading wv recursively from /tmp/tmp6jh4qoa_gensim_temp.wv.* with mmap=None
2017-06-25 14:56:02,626 : INFO : setting ignored attribute syn0norm to None
2017-06-25 14:56:02,627 : INFO : setting ignored attribute cum_table to None
2017-06-25 14:56:02,628 : INFO : loaded /tmp/tmp6jh4qoa_gensim_temp
2017-06-25 14:56:02,639 : INFO : collecting all words and their counts
2017-06-25 14:56:02,641 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-25 14:56:02,643 : INFO : collected 13 word types from a corpus of 13 raw words and 1 sentences
2017-06-25 14:56:02,644 : INFO : Updating model with new vocabulary
2017-06-25 14:56:02,647 : INFO : New added 0 unique words (0% of original 13)
                        and increased the count of 0 pre-existing words (0% of original 13)
2017-06-25 14:56:02,649 : INFO : deleting the raw counts dictionary o

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`Word2Vec` supports several word similarity tasks out of the box:

In [20]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

2017-06-25 14:56:03,953 : INFO : precomputing L2-norms of word weight vectors


[('stop', 0.9968920350074768)]

In [21]:
model.doesnt_match("input is lunch he sentence cat".split())

2017-06-25 14:56:04,564 : WARNING : vectors for words {'cat', 'lunch', 'input'} are not present in the model, ignoring these words


'sentence'

In [22]:
print(model.similarity('human', 'party'))
print(model.similarity('tree', 'murder'))

0.999276605162
0.996141322746


You can get the probability distribution for the center word given the context words as input:

In [23]:
print(model.predict_output_word(['emergency', 'beacon', 'received']))

[('more', 0.0010236123), ('can', 0.00089447433), ('continue', 0.00089196191), ('training', 0.00088589359), ('australia', 0.00075986236), ('it', 0.00074403704), ('could', 0.00073666498), ('when', 0.00073339697), ('united', 0.00073033909), ('man', 0.00073026447)]


The results here don't look good because the training corpus is very small. To get meaningful results one needs to train on 500k+ words.

If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [24]:
model['tree']  # raw NumPy vector of a word

array([-0.01903718,  0.00677472,  0.01860059,  0.01009176, -0.04003047,
       -0.02917769,  0.02238039,  0.02124756, -0.0334513 , -0.00827437,
        0.06820618, -0.05499271, -0.0474261 , -0.01810067, -0.07317127,
        0.02077067,  0.00568337, -0.02281303, -0.02411027, -0.00783623,
        0.07914981, -0.00492787,  0.0603894 ,  0.04410831, -0.01237845,
       -0.01142686,  0.00747105,  0.00931906, -0.00814121, -0.0424807 ,
       -0.06895473,  0.01451517,  0.05681378, -0.00269753,  0.01235684,
       -0.05537964,  0.04452592,  0.01428834,  0.04250315,  0.02652211,
        0.01692941,  0.03494624,  0.05407221, -0.08074032,  0.02800615,
       -0.03684049,  0.03077209, -0.00182472,  0.04108352, -0.03603134,
        0.01280163, -0.00281598, -0.05482334, -0.03404022,  0.00542461,
       -0.03681742, -0.00742   , -0.05267051,  0.01627189, -0.01766601,
        0.03193061,  0.02058678,  0.00358074, -0.05638171,  0.02366105,
        0.02153986, -0.05803887, -0.02922741,  0.00926538,  0.01

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example: Semantic- words like cat, dog, cow, etc. have a tendency to lie close by
         Syntactic- words like run, running or cut, cutting lie close together.
Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

Additional dependencies : 
- sklearn
- numpy
- plotly

The function below can be used to plot the embeddings in an ipython notebook.
It requires the model as the necessary parameter. If you don't have the model, you can load it by

`model = gensim.models.Word2Vec.load('path/to/model')`

If you don't want to plot inside a notebook, set the `plot_in_notebook` parameter to `False`.

Note: the model used for the visualisation is trained on a small corpus. Thus some of the relations might not be so clear

Beware : This sort dimension reduction comes at the cost of loss of information.

In [25]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [26]:
reduce_dimensions(model)

2017-06-25 14:56:19,386 : INFO : starting tSNE dimensionality reduction. This may take some time.
2017-06-25 14:56:32,335 : INFO : All done. Plotting.


## Conclusion

In this tutorial we learned how to train word2vec models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!

## Links


Full `word2vec` API docs [here](http://radimrehurek.com/gensim/models/word2vec.html); get [gensim](http://radimrehurek.com/gensim/) here. Original C toolkit and `word2vec` papers by Google [here](https://code.google.com/archive/p/word2vec/).